In [1]:
import csv
import glob
import os
try:
    code_dir
except NameError:
    code_dir = os.getcwd()
    out_dir = code_dir.replace("codes_local", "3_ChIP_peak_search")
    print(code_dir)

/Volumes/Huitian/Projects/CD8_DEV_SC/3_ChIP/codes_local


In [2]:
peaks_dir = "/Volumes/Huitian/Projects/T_Cell_ChIP/202012_ChIP/2_MACS2"
peaks_files = glob.glob("%s/*peaks.xls"%peaks_dir)

In [9]:
search_dict = {"chd7_region1": ["4", 8868977, 8875485],
              "chd7_region2": ["4", 8593648, 8595285],
              "chd7_region3": ["4", 9006987, 9020087],
               "chd7_region4": ["4", 8915702, 8918956],
               "irf4_region1": ["13", 30765801, 30766419],
               "nfatc1_region1": ["18", 80737497, 80793360]
              }

In [10]:
search_name = "nfatc1_region1"
search_chr, search_s, search_e = search_dict[search_name]

search_len = search_e - search_s
search_outname = out_dir + "/" + search_name + ".csv"

with open(search_outname, "w") as fout:
    wfout = csv.writer(fout, delimiter = ",")
    wfout.writerow(["chr", "start", "end", "length", 
                    "abs_summit", "pileup", "-log10pval",
                    "fold_enrichment", "-log10qval", "name"])
    for peaks_file in peaks_files:
        with open(peaks_file, "r") as fin:
            rfin = csv.reader(fin, delimiter = "\t")
            start_flag = False
            for row in rfin:
                if not start_flag:
                    if len(row) > 1 and row[0] == "chr":
                        start_flag = True
                else:
                    if str(row[0]) == search_chr:
                        row_s = int(row[1])
                        row_e = int(row[2])
                        row_len = row_e - row_s
                        if (search_len + row_len) > (max(row_e, search_e) - min(row_s, search_s)):
                            wfout.writerow(row)